In [2]:
import bisect
import math

import cv2
import h5py
import matplotlib.pyplot as plt
import numpy as np
import tqdm
import os
import folium

os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd


from shapely import Point

import jaxlie
import plotly.graph_objects as go  
import trajectory_utils


In [ ]:
# hdf5_dir = "2022-04-22-20230326T120319Z-002/2022-04-22/"
filename ="/home/sf_afn/Downloads/record_2023-04-24_13-49-28_rm7.hdf5"
sequence_f = h5py.File(filename, "r")

print("Keys: %s" % sequence_f.keys())
print("version",sequence_f.attrs['3DAI_version'])

In [ ]:
image_timestamps = trajectory_utils.get_first_last_timestamps(sequence_f)

In [ ]:
image_timestamps

In [ ]:
# Show one image

one_image = trajectory_utils.get_image_at_timestamp(sequence_f, 700,image_timestamps)
#print(type(one_image))
plt.figure()
plt.imshow(one_image)
plt.show()



In [ ]:
fused_positions = trajectory_utils.get_fused_positions(sequence_f)

In [ ]:
trajectory_utils.plot_2D_trajectory(fused_positions)


In [ ]:
trajectory_utils.plot_3D_trajectory(fused_positions)

In [ ]:
# fetch parked car detections from a csv from Emmanuel edited by Sylvain
file_csv = "/home/sf_afn/Downloads/parked_car_triangulations.csv"
parked_cars_gdf = gpd.read_file(file_csv)

In [ ]:
parked_cars_gdf.head()

In [ ]:
parked_cars_gdf.describe()

In [ ]:
# --->BK1 base map declare
# create a map centered on Sodermalm
bbox_center = [59.312345, 18.066072] 
map_1 = folium.Map(location= bbox_center, zoom_start=14,tiles="cartodbpositron" )

# tile = folium.TileLayer(
        # tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        # attr = 'Esri',
    #     # name = 'Esri Satellite',
    #     overlay = False,
    #     control = True
    #    ).add_to(map)

In [ ]:
map_1

In [4]:
permitted_parking_gdf = gpd.read_file('/home/sf_afn/Downloads/univrses_data/permitted_parking.json')

In [5]:
permitted_parking_gdf.head()

,id,FID,FEATURE_OBJECT_ID,FEATURE_VERSION_ID,EXTENT_NO,VALID_FROM,VALID_TO,VEHICLE,START_TIME,END_TIME,...,CITY_DISTRICT,PARKING_DISTRICT,OTHER_INFO,VF_METER,VF_PLATSER,VF_PLATS_TYP,ADDRESS,RDT_URL,PARKING_RATE,geometry
0,LTFR_P_TILLATEN.44817868,44817868,13742836,1,5,2018-01-29 23:00:00+00:00,NaT,fordon,1600.0,800.0,...,Bagarmossen,Bagarmossen,Servicetid fredag 08:00-16:00 1/11 - 15/5,NaN,NaN,"P Avgift, boende",Lagavägen 2,https://rdt.transportstyrelsen.se/rdt/AF06_Vie...,"taxa 5: 5 kr/tim vardagar 7-19, Boende: 300 kr...","LINESTRING (157288.172 6573698.371, 157287.080..."
1,LTFR_P_TILLATEN.44817869,44817869,13750219,1,1,2017-10-03 22:00:00+00:00,NaT,rörelsehindrade,NaN,NaN,...,Södermalm,Södermalm,NaN,NaN,3.0,Reserverad p-plats rörelsehindrad,Heleneborgsgatan 4A,https://rdt.transportstyrelsen.se/rdt/AF06_Vie...,"taxa 3: 20 kr/tim vardagar 7-19, 15 kr/tim dag...","LINESTRING (152484.638 6578284.974, 152483.502..."
2,LTFR_P_TILLATEN.44817870,44817870,13751474,1,1,2017-10-24 22:00:00+00:00,NaT,rörelsehindrade,NaN,NaN,...,Vasastaden,Vasastan,NaN,6.0,NaN,Reserverad p-plats rörelsehindrad,Observatoriegatan 20,https://rdt.transportstyrelsen.se/rdt/AF06_Vie...,"taxa 3: 20 kr/tim vardagar 7-19, 15 kr/tim dag...","LINESTRING (152736.834 6580534.810, 152731.302..."
3,LTFR_P_TILLATEN.44817871,44817871,13718834,1,1,2018-01-29 23:00:00+00:00,NaT,fordon,NaN,NaN,...,Bagarmossen,Bagarmossen,Servicetid tisdag 08:00-16:00 1/11 - 15/5,NaN,NaN,"P Avgift, boende",Letstigen 46 - Letstigen 56,https://rdt.transportstyrelsen.se/rdt/AF06_Vie...,"taxa 5: 5 kr/tim vardagar 7-19, Boende: 300 kr...","LINESTRING (158341.006 6573198.838, 158366.896..."
4,LTFR_P_TILLATEN.44817872,44817872,13718834,1,1,2018-01-29 23:00:00+00:00,NaT,fordon,1600.0,800.0,...,Bagarmossen,Bagarmossen,Servicetid tisdag 08:00-16:00 1/11 - 15/5,NaN,NaN,"P Avgift, boende",Letstigen 46 - Letstigen 56,https://rdt.transportstyrelsen.se/rdt/AF06_Vie...,"taxa 5: 5 kr/tim vardagar 7-19, Boende: 300 kr...","LINESTRING (158341.006 6573198.838, 158366.896..."


In [ ]:
permitted_parking_gdf.info()

In [ ]:
fused_positions[:10,1]

In [ ]:
permitted_parking_gdf.crs

In [ ]:
permitted_parkng_4326 = permitted_parking_gdf.to_crs(epsg=4326)

In [ ]:
permitted_parkng_4326.crs

In [ ]:
# list(permitted_parkng_4326['geometry'].iloc[190].xy[0], permitted_parkng_4326['geometry'].iloc[190].xy[1])

In [ ]:
# print(permitted_parkng_4326['geometry'].iloc[190].coords)



In [ ]:
permitted_parkng_4326['xy']= permitted_parkng_4326['geometry'].apply(lambda row: row.xy )

In [ ]:
permitted_parkng_4326['xy'].iloc[10][0]

In [ ]:
permitted_parkng_4326['location_coords'] = permitted_parkng_4326['xy'].apply(lambda row: [(x,y) for x,y in zip(row[1],row[0])])

In [ ]:
permitted_parkng_4326['location_coords'].iloc[109]

In [ ]:
# ----> BK2 permitted parking polylines
# add geometry to a feature group
ls_fg = folium.FeatureGroup(name= "permitted-parking")
#polyline held as child of the map
for linestring in permitted_parkng_4326['location_coords']:
    ls_fg.add_child(folium.PolyLine(locations= linestring))
# feature group added to the child
# warning: add layer control after adding your last layer otherwise nothing will work as its supposed to!
map_1.add_child(ls_fg)


In [ ]:
positions = trajectory_utils.get_positions_geolocation(sequence_f)

In [ ]:
positions[1]

In [ ]:
traj_gdf = gpd.GeoDataFrame(positions)

In [ ]:
traj_gdf.describe()

In [ ]:
traj_gdf.head()

In [ ]:
traj_gdf.columns

In [ ]:
traj_gdf['geometry'] = traj_gdf.apply(lambda row: Point(row['latitude'], row['longitude']), axis= 1)

In [ ]:
traj_gdf.head()

In [ ]:
traj_gdf.explore()

In [ ]:
list(traj_gdf['geometry'][0].coords)[0]

In [ ]:
# ----> BK 3 route cirlcle markers
# adding route as circle markers
rcm_fg = folium.FeatureGroup(name= "route_circlemarkers")

for point in traj_gdf['geometry']:
    rcm_fg.add_child(folium.CircleMarker(location= list(point.coords)[0], radius= 2, color= 'red'))

map_1.add_child(rcm_fg)


In [ ]:
map_1

In [ ]:
traj_gdf.apply(lambda row: list(row['geometry'].coords[0]), axis= 1)

In [ ]:
# ---> BK 4 route polylines
# adding route as a polyline

pl_fg = folium.FeatureGroup(name= "route_polyline")


pl_fg.add_child(folium.PolyLine(locations= traj_gdf.apply(lambda row: list(row['geometry'].coords[0]), axis= 1), color= 'green'))

map_1.add_child(pl_fg)

#folium.LayerControl().add_to(map_1)


In [ ]:
map_1

In [ ]:

parked_cars_gdf.head()

In [ ]:
parked_cars_gdf.crs

In [ ]:
parked_cars_gdf.set_crs('espg:4326')

In [ ]:
parked_cars_gdf['geometry'] = parked_cars_gdf.apply(lambda row: Point(row['latitude'],row['longitude'] ), axis = 1)

In [ ]:
# ---> BK 5 parking detections 
pcm_fg = folium.FeatureGroup(name= "parked_cars_triangulation")

#pcm_fg.add_child(folium.CircleMarker(location= parked_cars_gdf.apply(lambda row: row['geometry'].coords[0], axis=1), color = 'green'))
for point in parked_cars_gdf['geometry']:
    pcm_fg.add_child(folium.CircleMarker(location= list(point.coords)[0], radius= 2, color= 'orange'))


map_1.add_child(pcm_fg)


In [ ]:
folium.LayerControl().add_to(map_1)

In [ ]:
map_1

In [ ]:
permitted_parkng_4326.head()

In [ ]:
traj_gdf.head()

In [ ]:
line = permitted_parkng_4326['location_coords'].iloc[1]
print(line)

In [ ]:
list(line.coords)

In [ ]:
point = traj_gdf['geometry'].iloc[1]
print(point)

In [ ]:
point_coords = point.x, point.y
point_coords

In [ ]:
# find distance between point and line
dist = line.distance(point)
dist

In [ ]:
# find a point on the line nearest to the external point
near_point = line.interpolate(line.project(point))

In [ ]:
near_point_coords = near_point.y, near_point.x
near_point_coords

In [ ]:
bbox_center = [59.312345, 18.066072] 
map_2 = folium.Map(location= bbox_center, zoom_start=14,tiles="cartodbpositron" )

In [ ]:
folium.CircleMarker(location= near_point_coords, color = 'red').add_to(map_2)
folium.CircleMarker(location= point_coords, color = 'green').add_to(map_2)
folium.PolyLine(locations= permitted_parkng_4326['location_coords'], color= 'blue').add_to(map_2)


In [ ]:
map_2